In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

from pathlib import Path
import os
import sys

# Setup for importing custom modules.
sys.path.insert(0, str(Path(os.getcwd()).parent.parent))

$$
\begin{align*}
\frac {\partial}{\partial t} \left(\begin{align*} u \\ v \end{align*}\right)
    &=
    \left( \begin{align*} &\lambda (A) &-\omega (A) \\ &\omega (A) &\lambda (A) \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right) \\
    &=
    \left( \begin{align*} &1 - A^2 &\beta A^2 \\ &-\beta A^2 & 1 - A^2 \end{align*} \right)
    \left( \begin{align*} u \\ v \end{align*} \right)
    + D \nabla^2 \left( \begin{align*} u \\ v \end{align*} \right)
\end{align*}
$$
with $A^2 = u^2 + v^2$.
Furthermore this results in the following system
$$
U_t = \lambda(A)U - \omega(A)V + D_1 \nabla^2 U \\
V_t = \omega(A)U + \lambda(A)V + D_2 \nabla^2 V
$$
plugging in what we have for $\lambda(A) = 1 - A^2$ and $\omega(A) = -\beta A^2$, gives us
$$
\begin{align*}
U_t &= (1 - A^2) U + \beta A^2 V + D_1 \nabla^2 U \\
V_t &= - \beta A^2 U + (1 - A^2) V + D_2 \nabla^2 V.
\end{align*}
$$
We can also advance things further by plugging in our values of $\beta$ and $A^2$
$$
\begin{align*}
U_t &= U - U^3 + V^2 U + \beta U^2 V + V^3 + D_1 \nabla^2 U \\
V_t &= - \beta U^3 -\beta V^2 U + V - U^2 v + V^3 + D_2 \nabla^2 V.
\end{align*}
$$
Finally the second gradient operator is given by $\nabla^2 = \partial_x^2 + \partial_y^2$, therefore we have
$$
\begin{align*}
U_t &= U - U^3 + V^2 U + \beta U^2 V + V^3 + D_1 ( \partial_x^2 U + \partial_y^2 U ) \\
V_t &= - \beta U^3 -\beta V^2 U + V - U^2 v + V^3 + D_2 ( \partial_x^2 V + \partial_y^2 V ).
\end{align*}
$$